## Q&A
1. ALS都有哪些应用场景: ALS的矩阵分解算法常应用于推荐系统中，将用户(user)对商品(item)的评分矩阵，分解为用户对商品隐含特征的偏好矩阵，和商品在隐含特征上的映射矩阵, 因此，主要有两部分 1） 评分预测分析，用户对商品合适程度,喜好程度.用户和商品就组成了一个矩阵,只要用户点击了商品,就对这个商品有个评分了,而有的却没有点击,它是空白的,我们要做的就是填充这些空白,在空白处根据之前的权重预测一个评分。 2） topN 归类分析，和rank 类似，但主要是为了实现最主要的 top N 归类，实现协同过滤

2. ALS进行矩阵分解的时候，为什么可以并行化处理: ALS 在进行特征内容矩阵分解时只会依赖于自身项，不依赖于其他用户的特征向量，一样可以将不同用户的更新公式放到不同的服务器上执行，因此可以进行并行处理

3. 梯度下降法中的批量梯度下降（BGD），随机梯度下降（SGD），和小批量梯度下降有什么区别（MBGD:  批量梯度下降法是最原始的形式，它是指在每一次迭代时使用所有样本来进行梯度的更新.即批量的完成所有梯度下降。 随机梯度下降是每次迭代使用一个样本来对参数进行更新，因此更加快速。 小批量梯度下降，是对批量梯度下降以及随机梯度下降的一个折中，每次迭代使用 batch_size 个样本来对参数进行更新，内存利用率会提升。

In [4]:
#from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold
# import pandas as pd
#without titles
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./ratings.csv', reader=reader)


In [6]:
#build_full_trainset()方法可用于在整个训练集上训练
train_set = data.build_full_trainset()

# ALS优化，用字典的形式表示，n_epoch 是迭代次数，reg_u 是uesr项的正则化系数默认值为15，reg_i 是item 项的正则化系数默认值为10(https://surprise.readthedocs.io/en/stable/prediction_algorithms.html?highlight=reg_u#baselines-estimates-configuration)
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
# SGD优化
#bsl_options = {'method': 'sgd','n_epochs': 5}
algo = BaselineOnly(bsl_options=bsl_options)


In [9]:
#algo = BaselineOnly()
#基于正太分布预测随机评分 
#algo = NormalPredictor()

# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
# raw user id (as in the ratings file). They are **strings**!
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = algo.predict(uid, iid, r_ui=4, verbose=True)
pred

Estimating biases using als...
RMSE: 0.8662
Estimating biases using als...
RMSE: 0.8625
Estimating biases using als...
RMSE: 0.8632
user: 196        item: 302        r_ui = 4.00   est = 4.18   {'was_impossible': False}


Prediction(uid='196', iid='302', r_ui=4, est=4.181439040615394, details={'was_impossible': False})

In [12]:
#KNN 
from surprise import KNNBasic, NormalPredictor
sim_options = {'name': 'cosine','n_epochs': 5,'user_based': True}
algo = KNNBasic(sim_options=sim_options)
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid, r_ui=4, verbose=True)
pred

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9544
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9539
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9529
user: 196        item: 302        r_ui = 4.00   est = 3.72   {'actual_k': 40, 'was_impossible': False}


Prediction(uid='196', iid='302', r_ui=4, est=3.71777688829686, details={'actual_k': 40, 'was_impossible': False})

In [13]:
import random

# 定语从句语法
grammar = '''
战斗 => 施法  ， 结果 。
施法 => 主语 动作 技能 
结果 => 主语 获得 效果
主语 => 张飞 | 关羽 | 赵云 | 典韦 | 许褚 | 刘备 | 黄忠 | 曹操 | 鲁班七号 | 貂蝉
动作 => 施放 | 使用 | 召唤 
技能 => 一骑当千 | 单刀赴会 | 青龙偃月 | 刀锋铁骑 | 黑暗潜能 | 画地为牢 | 守护机关 | 狂兽血性 | 龙鸣 | 惊雷之龙 | 破云之龙 | 天翔之龙
获得 => 损失 | 获得 
效果 => 数值 状态
数值 => 1 | 1000 |5000 | 100 
状态 => 法力 | 生命
'''

# 得到语法字典
def getGrammarDict(gram, linesplit = "\n", gramsplit = "=>"):
    result = {}

    for line in gram.split(linesplit):

        if not line.strip(): 
            continue
        expr, statement = line.split(gramsplit)
        result[expr.strip()] = [i.split() for i in statement.split("|")]
    #print(result)
    return result

# 生成句子
def generate(gramdict, target, isEng = False):
    if target not in gramdict: 
        return target
    find = random.choice(gramdict[target])

    blank = ''

    if isEng: 
        blank = ' '
    return blank.join(generate(gramdict, t, isEng) for t in find)

gramdict = getGrammarDict(grammar)
print(generate(gramdict,"战斗"))
print(generate(gramdict,"战斗", True))

曹操使用画地为牢，张飞损失100生命。
黄忠 施放 破云之龙 ， 鲁班七号 损失 1 生命 。


In [14]:
print(generate(gramdict,"施法"))

黄忠召唤惊雷之龙
